In [1]:
import pandas as pd
import numpy as np
import gspread
from datetime import date
from datetime import datetime
from datetime import datetime, timedelta
from pandas import json_normalize

import re

import sys                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  
sys.path.append('../')

import zoho_crm_api_module as crm

import sys
sys.path.append('../../../zoho_data_app/')

# importing
import main_module as m

%load_ext autoreload
%autoreload 2

In [2]:
gc = gspread.oauth(
    credentials_filename= r'C:\Users\fajar.fatoni\Documents\Python\Data\Google Credentials\karma-oauth.json'
)

SHEET_ID = '1_zOrWK1gAo-tHDsKqU6AaqVzi4xaNpOELDwtbBx5F6M'
SHEET_NAME = 'Karma Beach - LIVE'


cols = [
    "id","Created_Time", "First_Name", "Last_Name", "Email", "Birthdate", "Phone", 
    "Street", "City", "Country", "Brand", "Lead_Sub_Brand", "Lead_Source", 
    "Lead_Source_Description", "Lead_Locations", "Lead_History", "Website", 
    "Card_Number", "Card_Expired", "Do_you_live_in_Bali"
]

live_cols = [
    "Record Id", "Created Time", "First Name", "Last Name", "Email", "Birthdate", 
    "Phone", "Street", "City", "Country", "Lead Brand", "Lead Sub-Brand", 
    "Lead Source", "Lead Source Description", "Lead Locations", "Lead History", 
    "Website", "Card Number", "Card Expired", "Do you live in Bali?"
]

# Function to fix the encoding
def fix_encoding(garbled_string):
    try:
        # Encode the garbled string to bytes using 'latin1' and then decode it to 'utf-8'
        return garbled_string.encode('latin1').decode('utf-8')
    except Exception as e:
        return garbled_string  # Return the original string if there's an error


In [3]:
crm_token = crm.get_crm_token()

In [4]:
df = crm.get_kbb_vip_data(crm_token, cols)
df.fillna('empty', inplace=True)
new_df = crm.get_kbb_vip_filter(df)
new_df['id'] = 'zcrm_'+new_df['id']
new_df.rename(columns={'id': 'Record Id'}, inplace=True)
new_df['Created_Time'] = new_df['Created_Time'].str.replace('T', ' ').str.replace(r'\+\d{2}:\d{2}', '', regex=True)
new_df['Created_Time']  = pd.to_datetime(new_df['Created_Time']).dt.strftime('%d/%m/%Y')
new_df.loc[new_df['Card_Expired'] == 'empty', 'Card_Expired'] = '1999-09-09'
new_df['Card_Expired']  = pd.to_datetime(new_df['Card_Expired']).dt.strftime('%d/%m/%Y')
new_df['First_Name'] = new_df['First_Name'].apply(fix_encoding)
new_df['Last_Name'] = new_df['Last_Name'].apply(fix_encoding)
m.clean_number(new_df, 'Phone')
m.replace_not_valid_to_empty(new_df, 'Phone')
# new_df = df.copy()
new_df.sort_values('Created_Time', ascending=False, inplace=True)
new_df.reset_index(drop=True, inplace=True)
count_data_awal = new_df.shape[0]
new_df

c:\Users\fajar.fatoni\Documents\Python\Task Offline\zoho_data_app\zoho_api\sync_module\..\zoho_crm_api_module.py:267: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  concat_df['Lead_Locations'].replace('\[|\]|\'','', regex=True, inplace=True)
c:\Users\fajar.fatoni\Documents\Python\Task Offline\zoho_data_app\zoho_api\sync_module\../../../zoho_data_app\main_module.py:104: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pan

,Record Id,Created_Time,First_Name,Last_Name,Email,Birthdate,Phone,Street,City,Country,Brand,Lead_Sub_Brand,Lead_Source,Lead_Source_Description,Lead_Locations,Lead_History,Website,Card_Number,Card_Expired,Do_you_live_in_Bali
0,zcrm_3584690000572744001,30/10/2025,Artha,Wanodya,arthawanodya@gmail.com,empty,6282182422800,empty,empty,Indonesia,Karma Resorts,Karma Beach,Karma Beach VIP 2023,Karma Beach VIP membership sign up 2023,empty,empty,https://karmagroup.com/karma-beach-vip/,2025001000300003,30/10/2026,False
1,zcrm_3584690000572709031,30/10/2025,Alex,McConnell,alextwine22@gmail.com,empty,61430296825,empty,empty,Australia,Karma Resorts,Karma Beach,Karma Beach VIP 2023,Karma Beach VIP membership sign up 2023,empty,empty,https://karmagroup.com/karma-beach-vip/,2025001000300002,30/10/2026,False
2,zcrm_3584690000572709001,30/10/2025,Natallia,Chumakova,go@royaltravel.by,empty,375293553464,empty,empty,Belarus,Karma Resorts,Karma Beach,Karma Beach VIP 2023,Karma Beach VIP membership sign up 2023,empty,empty,https://karmagroup.com/karma-beach-vip/,2025001000300001,30/10/2026,False
3,zcrm_3584690000572567001,28/10/2025,Lino,Scidone,lino.scidone@gmail.com,empty,61425757400,empty,empty,Australia,Karma Resorts,Karma Beach,Karma Beach VIP 2023,Karma Beach VIP membership sign up 2023,empty,empty,https://karmagroup.com/karma-beach-vip/,2025001000280004,28/10/2026,False
4,zcrm_3584690000572538001,28/10/2025,Luciana,Begazo,collabsluciana@gmail.com,empty,34613463996,empty,empty,Spain,Karma Resorts,Karma Beach,Karma Beach VIP 2023,Karma Beach VIP membership sign up 2023,empty,empty,https://karmagroup.com/karma-beach-vip/,2025001000280003,28/10/2026,False
5,zcrm_3584690000572527031,28/10/2025,Bernardo,Godecke,bernardo.godecke@gmail.com,empty,34662944062,empty,empty,Spain,Karma Resorts,Karma Beach,Karma Beach VIP 2023,Karma Beach VIP membership sign up 2023,empty,empty,https://karmagroup.com/karma-beach-vip/,2025001000280002,28/10/2026,True
6,zcrm_3584690000572508070,28/10/2025,Mikhail,Kulakov,kulakov130620@ya.ru,empty,6282341820854,empty,empty,Indonesia,Karma Resorts,Karma Beach,Karma Beach VIP 2023,Karma Beach VIP membership sign up 2023,empty,empty,https://karmagroup.com/karma-beach-vip/,2025001000280001,28/10/2026,True
7,zcrm_3584690000572318166,27/10/2025,Valentina,Vrech,valentinavrech@hotmail.com,empty,31653858650,empty,empty,Netherlands,Karma Resorts,Karma Beach,Karma Beach VIP 2023,Karma Beach VIP membership sign up 2023,empty,empty,https://karmagroup.com/karma-beach-vip/,2025001000270001,27/10/2026,False
8,zcrm_3584690000572431001,27/10/2025,Charlotte,Cole,charlotte@spectrumsc.com.au,empty,61408119994,empty,empty,Australia,Karma Resorts,Karma Beach,Karma Beach VIP 2023,Karma Beach VIP membership sign up 2023,empty,empty,https://karmagroup.com/karma-beach-vip/,2025001000270003,27/10/2026,False
9,zcrm_3584690000572312139,27/10/2025,OLGA,KUNASHOVA,kunashova.ol@yandex.ru,empty,628133663148,empty,empty,Indonesia,Karma Resorts,Karma Beach,Karma Beach VIP 2023,Karma Beach VIP membership sign up 2023,empty,empty,https://karmagroup.com/karma-beach-vip/,2025001000270002,27/10/2026,True


In [5]:
new_df['First_Name'] = new_df['First_Name'].str.title()
new_df['Last_Name'] = new_df['Last_Name'].str.title()
new_df['Email'] = m.lowercase(new_df, 'Email')
m.clean_space(new_df, 'Email')

new_df
# new_df.to_excel(r"C:\Users\fajar.fatoni\Documents\Python\Data\kbbvip_live.xlsx", index=False)

c:\Users\fajar.fatoni\Documents\Python\Task Offline\zoho_data_app\zoho_api\sync_module\../../../zoho_data_app\main_module.py:127: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[df_columns].replace(' ','', regex=True, inplace=True)


,Record Id,Created_Time,First_Name,Last_Name,Email,Birthdate,Phone,Street,City,Country,Brand,Lead_Sub_Brand,Lead_Source,Lead_Source_Description,Lead_Locations,Lead_History,Website,Card_Number,Card_Expired,Do_you_live_in_Bali
0,zcrm_3584690000572744001,30/10/2025,Artha,Wanodya,arthawanodya@gmail.com,empty,6282182422800,empty,empty,Indonesia,Karma Resorts,Karma Beach,Karma Beach VIP 2023,Karma Beach VIP membership sign up 2023,empty,empty,https://karmagroup.com/karma-beach-vip/,2025001000300003,30/10/2026,False
1,zcrm_3584690000572709031,30/10/2025,Alex,Mcconnell,alextwine22@gmail.com,empty,61430296825,empty,empty,Australia,Karma Resorts,Karma Beach,Karma Beach VIP 2023,Karma Beach VIP membership sign up 2023,empty,empty,https://karmagroup.com/karma-beach-vip/,2025001000300002,30/10/2026,False
2,zcrm_3584690000572709001,30/10/2025,Natallia,Chumakova,go@royaltravel.by,empty,375293553464,empty,empty,Belarus,Karma Resorts,Karma Beach,Karma Beach VIP 2023,Karma Beach VIP membership sign up 2023,empty,empty,https://karmagroup.com/karma-beach-vip/,2025001000300001,30/10/2026,False
3,zcrm_3584690000572567001,28/10/2025,Lino,Scidone,lino.scidone@gmail.com,empty,61425757400,empty,empty,Australia,Karma Resorts,Karma Beach,Karma Beach VIP 2023,Karma Beach VIP membership sign up 2023,empty,empty,https://karmagroup.com/karma-beach-vip/,2025001000280004,28/10/2026,False
4,zcrm_3584690000572538001,28/10/2025,Luciana,Begazo,collabsluciana@gmail.com,empty,34613463996,empty,empty,Spain,Karma Resorts,Karma Beach,Karma Beach VIP 2023,Karma Beach VIP membership sign up 2023,empty,empty,https://karmagroup.com/karma-beach-vip/,2025001000280003,28/10/2026,False
5,zcrm_3584690000572527031,28/10/2025,Bernardo,Godecke,bernardo.godecke@gmail.com,empty,34662944062,empty,empty,Spain,Karma Resorts,Karma Beach,Karma Beach VIP 2023,Karma Beach VIP membership sign up 2023,empty,empty,https://karmagroup.com/karma-beach-vip/,2025001000280002,28/10/2026,True
6,zcrm_3584690000572508070,28/10/2025,Mikhail,Kulakov,kulakov130620@ya.ru,empty,6282341820854,empty,empty,Indonesia,Karma Resorts,Karma Beach,Karma Beach VIP 2023,Karma Beach VIP membership sign up 2023,empty,empty,https://karmagroup.com/karma-beach-vip/,2025001000280001,28/10/2026,True
7,zcrm_3584690000572318166,27/10/2025,Valentina,Vrech,valentinavrech@hotmail.com,empty,31653858650,empty,empty,Netherlands,Karma Resorts,Karma Beach,Karma Beach VIP 2023,Karma Beach VIP membership sign up 2023,empty,empty,https://karmagroup.com/karma-beach-vip/,2025001000270001,27/10/2026,False
8,zcrm_3584690000572431001,27/10/2025,Charlotte,Cole,charlotte@spectrumsc.com.au,empty,61408119994,empty,empty,Australia,Karma Resorts,Karma Beach,Karma Beach VIP 2023,Karma Beach VIP membership sign up 2023,empty,empty,https://karmagroup.com/karma-beach-vip/,2025001000270003,27/10/2026,False
9,zcrm_3584690000572312139,27/10/2025,Olga,Kunashova,kunashova.ol@yandex.ru,empty,628133663148,empty,empty,Indonesia,Karma Resorts,Karma Beach,Karma Beach VIP 2023,Karma Beach VIP membership sign up 2023,empty,empty,https://karmagroup.com/karma-beach-vip/,2025001000270002,27/10/2026,True


In [6]:
# # mencari tanggal 8 hari kebelakang
# calculate_date = datetime.now() - timedelta(days=8)
# new_cal_date = str(calculate_date.strftime("%Y-%m-%d")).split(' ')
# first_date = new_cal_date[0]

# # mencari tanggal hari ini
# today = datetime.now() 
# second_date = today.strftime("%Y-%m-%d")

# print(first_date)
# print(second_date)

In [7]:
new_df.replace(['empty','09/09/1999'], '', inplace=True)
new_df

,Record Id,Created_Time,First_Name,Last_Name,Email,Birthdate,Phone,Street,City,Country,Brand,Lead_Sub_Brand,Lead_Source,Lead_Source_Description,Lead_Locations,Lead_History,Website,Card_Number,Card_Expired,Do_you_live_in_Bali
0,zcrm_3584690000572744001,30/10/2025,Artha,Wanodya,arthawanodya@gmail.com,,6282182422800,,,Indonesia,Karma Resorts,Karma Beach,Karma Beach VIP 2023,Karma Beach VIP membership sign up 2023,,,https://karmagroup.com/karma-beach-vip/,2025001000300003,30/10/2026,False
1,zcrm_3584690000572709031,30/10/2025,Alex,Mcconnell,alextwine22@gmail.com,,61430296825,,,Australia,Karma Resorts,Karma Beach,Karma Beach VIP 2023,Karma Beach VIP membership sign up 2023,,,https://karmagroup.com/karma-beach-vip/,2025001000300002,30/10/2026,False
2,zcrm_3584690000572709001,30/10/2025,Natallia,Chumakova,go@royaltravel.by,,375293553464,,,Belarus,Karma Resorts,Karma Beach,Karma Beach VIP 2023,Karma Beach VIP membership sign up 2023,,,https://karmagroup.com/karma-beach-vip/,2025001000300001,30/10/2026,False
3,zcrm_3584690000572567001,28/10/2025,Lino,Scidone,lino.scidone@gmail.com,,61425757400,,,Australia,Karma Resorts,Karma Beach,Karma Beach VIP 2023,Karma Beach VIP membership sign up 2023,,,https://karmagroup.com/karma-beach-vip/,2025001000280004,28/10/2026,False
4,zcrm_3584690000572538001,28/10/2025,Luciana,Begazo,collabsluciana@gmail.com,,34613463996,,,Spain,Karma Resorts,Karma Beach,Karma Beach VIP 2023,Karma Beach VIP membership sign up 2023,,,https://karmagroup.com/karma-beach-vip/,2025001000280003,28/10/2026,False
5,zcrm_3584690000572527031,28/10/2025,Bernardo,Godecke,bernardo.godecke@gmail.com,,34662944062,,,Spain,Karma Resorts,Karma Beach,Karma Beach VIP 2023,Karma Beach VIP membership sign up 2023,,,https://karmagroup.com/karma-beach-vip/,2025001000280002,28/10/2026,True
6,zcrm_3584690000572508070,28/10/2025,Mikhail,Kulakov,kulakov130620@ya.ru,,6282341820854,,,Indonesia,Karma Resorts,Karma Beach,Karma Beach VIP 2023,Karma Beach VIP membership sign up 2023,,,https://karmagroup.com/karma-beach-vip/,2025001000280001,28/10/2026,True
7,zcrm_3584690000572318166,27/10/2025,Valentina,Vrech,valentinavrech@hotmail.com,,31653858650,,,Netherlands,Karma Resorts,Karma Beach,Karma Beach VIP 2023,Karma Beach VIP membership sign up 2023,,,https://karmagroup.com/karma-beach-vip/,2025001000270001,27/10/2026,False
8,zcrm_3584690000572431001,27/10/2025,Charlotte,Cole,charlotte@spectrumsc.com.au,,61408119994,,,Australia,Karma Resorts,Karma Beach,Karma Beach VIP 2023,Karma Beach VIP membership sign up 2023,,,https://karmagroup.com/karma-beach-vip/,2025001000270003,27/10/2026,False
9,zcrm_3584690000572312139,27/10/2025,Olga,Kunashova,kunashova.ol@yandex.ru,,628133663148,,,Indonesia,Karma Resorts,Karma Beach,Karma Beach VIP 2023,Karma Beach VIP membership sign up 2023,,,https://karmagroup.com/karma-beach-vip/,2025001000270002,27/10/2026,True


In [8]:
import pandas as pd
# Assuming gc, SHEET_ID, SHEET_NAME, and fix_encoding are defined

df_live = pd.DataFrame() # Initialize an empty DataFrame

try:
    # Get data from Google Sheets
    spreadsheet = gc.open_by_key(SHEET_ID)
    worksheet = spreadsheet.worksheet(SHEET_NAME)
    # get_all_records assumes the first row is a header
    rows = worksheet.get_all_records()

    # Check if any data was returned
    if not rows:
        print(f"No data found in sheet '{SHEET_NAME}' or it lacks a header row.")
        # Define expected columns for an empty DataFrame if needed downstream
        df_live = pd.DataFrame(columns=['Card Expired', 'First Name', 'Last Name'])
    else:
        # Create DataFrame from the rows
        df_live = pd.DataFrame(rows)

        # --- Process columns only if they exist ---

        # Process 'Card Expired'
        if 'Card Expired' in df_live.columns:
            # Convert 'Card Expired' to datetime, handling potential errors
            df_live['Card Expired'] = pd.to_datetime(
                df_live['Card Expired'],
                dayfirst=True,
                errors='coerce' # Convert unparseable dates to NaT (Not a Time)
            )
            # Format back to string, handling NaT values gracefully
            df_live['Card Expired'] = df_live['Card Expired'].dt.strftime('%d/%m/%Y').fillna('') # Replace NaT with empty string
        else:
            print("Warning: Column 'Card Expired' not found in the sheet.")

        # Process 'First Name'
        if 'First Name' in df_live.columns:
            df_live['First Name'] = df_live['First Name'].apply(fix_encoding)
        else:
            print("Warning: Column 'First Name' not found in the sheet.")

        # Process 'Last Name'
        if 'Last Name' in df_live.columns:
            df_live['Last Name'] = df_live['Last Name'].apply(fix_encoding)
        else:
            print("Warning: Column 'Last Name' not found in the sheet.")

except Exception as e:
    print(f"An error occurred while accessing or processing the Google Sheet: {e}")
    # Ensure df_live is an empty DataFrame in case of error during processing
    if not isinstance(df_live, pd.DataFrame) or df_live.empty:
       df_live = pd.DataFrame(columns=['Card Expired', 'First Name', 'Last Name']) # Or just pd.DataFrame()
    print("DataFrame might be empty or partially processed due to the error.")


# Display the DataFrame
print("\nFinal DataFrame contents:")
# Use display() in environments like Jupyter/Colab, otherwise use print()
try:
    display(df_live)
except NameError:
    print(df_live)



Final DataFrame contents:


,Record Id,Created Time,First Name,Last Name,Email,Birthdate,Phone,Street,City,Country,Lead Brand,Lead Sub-Brand,Lead Source,Lead Source Description,Lead Locations,Lead History,Website,Card Number,Card Expired,Do you live in Bali?
0,zcrm_3584690000572567001,28/10/2025,Lino,Scidone,lino.scidone@gmail.com,,61425757400,,,Australia,Karma Resorts,Karma Beach,Karma Beach VIP 2023,Karma Beach VIP membership sign up 2023,,,https://karmagroup.com/karma-beach-vip/,2025001000280004,28/10/2026,FALSE
1,zcrm_3584690000572527031,28/10/2025,Bernardo,Godecke,bernardo.godecke@gmail.com,,34662944062,,,Spain,Karma Resorts,Karma Beach,Karma Beach VIP 2023,Karma Beach VIP membership sign up 2023,,,https://karmagroup.com/karma-beach-vip/,2025001000280002,28/10/2026,TRUE
2,zcrm_3584690000572508070,28/10/2025,Mikhail,Kulakov,kulakov130620@ya.ru,,6282341820854,,,Indonesia,Karma Resorts,Karma Beach,Karma Beach VIP 2023,Karma Beach VIP membership sign up 2023,,,https://karmagroup.com/karma-beach-vip/,2025001000280001,28/10/2026,TRUE
3,zcrm_3584690000572538001,28/10/2025,Luciana,Begazo,collabsluciana@gmail.com,,34613463996,,,Spain,Karma Resorts,Karma Beach,Karma Beach VIP 2023,Karma Beach VIP membership sign up 2023,,,https://karmagroup.com/karma-beach-vip/,2025001000280003,28/10/2026,FALSE
4,zcrm_3584690000572318166,27/10/2025,Valentina,Vrech,valentinavrech@hotmail.com,,31653858650,,,Netherlands,Karma Resorts,Karma Beach,Karma Beach VIP 2023,Karma Beach VIP membership sign up 2023,,,https://karmagroup.com/karma-beach-vip/,2025001000270001,27/10/2026,FALSE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10393,zcrm_3584690000007840074,02/04/2019,Oreste,Lentidoro,lentidoro@gmail.com,,61410699788,,,Australia,Karma Resorts,Karma Beach,Karma Beach VIP 2023,Karma Beach VIP membership sign up 2023,,Other - Other,https://karmagroup.com/karma-beach-vip/,2024000300100005,10/03/2025,FALSE
10394,zcrm_3584690000008059311,02/04/2019,Meylina,Ardiningsih,meylinaardi@gmail.com,,6281385951854,,,Indonesia,Karma Resorts,Karma Beach,Karma Beach VIP 2023,Karma Beach VIP membership sign up 2023,,Other - Other,https://karmagroup.com/karma-beach-vip/,2024000100200010,20/01/2025,FALSE
10395,zcrm_3584690000008362510,02/04/2019,Caslin,Liu,Caslinliu@yahoo.com.sg,,6591192897,,,Singapore,Karma Resorts,Other,Past Guests,Karma Beach VIP membership sign up 2023,Karma Kandara,Other Bali DB - Purchased,https://karmagroup.com/karma-beach-vip/,2024000300240002,24/03/2025,FALSE
10396,zcrm_3584690000007896028,02/04/2019,Wesley,Powell,wez.powell@gmail.com,,61459554763,,,Australia,Karma Resorts,Karma Beach,Karma Beach VIP 2023,Karma Beach VIP membership sign up 2023,,Other - Other,https://karmagroup.com/karma-beach-vip/,2024000500240018,24/05/2025,FALSE


In [9]:
df_update = new_df.loc[~new_df['Record Id'].isin(df_live['Record Id'])].copy()
df_update

,Record Id,Created_Time,First_Name,Last_Name,Email,Birthdate,Phone,Street,City,Country,Brand,Lead_Sub_Brand,Lead_Source,Lead_Source_Description,Lead_Locations,Lead_History,Website,Card_Number,Card_Expired,Do_you_live_in_Bali
0,zcrm_3584690000572744001,30/10/2025,Artha,Wanodya,arthawanodya@gmail.com,,6282182422800,,,Indonesia,Karma Resorts,Karma Beach,Karma Beach VIP 2023,Karma Beach VIP membership sign up 2023,,,https://karmagroup.com/karma-beach-vip/,2025001000300003,30/10/2026,False
1,zcrm_3584690000572709031,30/10/2025,Alex,Mcconnell,alextwine22@gmail.com,,61430296825,,,Australia,Karma Resorts,Karma Beach,Karma Beach VIP 2023,Karma Beach VIP membership sign up 2023,,,https://karmagroup.com/karma-beach-vip/,2025001000300002,30/10/2026,False
2,zcrm_3584690000572709001,30/10/2025,Natallia,Chumakova,go@royaltravel.by,,375293553464,,,Belarus,Karma Resorts,Karma Beach,Karma Beach VIP 2023,Karma Beach VIP membership sign up 2023,,,https://karmagroup.com/karma-beach-vip/,2025001000300001,30/10/2026,False


In [10]:
df_update.sort_values(by=['Created_Time'], inplace=True, ascending=False)
rename_mapping = dict(zip(cols, live_cols))
df_update = df_update.rename(columns=rename_mapping)
df_update.reset_index(drop=True, inplace=True)
df_update

count_df_update = df_update.shape[0]


In [11]:
df_live

,Record Id,Created Time,First Name,Last Name,Email,Birthdate,Phone,Street,City,Country,Lead Brand,Lead Sub-Brand,Lead Source,Lead Source Description,Lead Locations,Lead History,Website,Card Number,Card Expired,Do you live in Bali?
0,zcrm_3584690000572567001,28/10/2025,Lino,Scidone,lino.scidone@gmail.com,,61425757400,,,Australia,Karma Resorts,Karma Beach,Karma Beach VIP 2023,Karma Beach VIP membership sign up 2023,,,https://karmagroup.com/karma-beach-vip/,2025001000280004,28/10/2026,FALSE
1,zcrm_3584690000572527031,28/10/2025,Bernardo,Godecke,bernardo.godecke@gmail.com,,34662944062,,,Spain,Karma Resorts,Karma Beach,Karma Beach VIP 2023,Karma Beach VIP membership sign up 2023,,,https://karmagroup.com/karma-beach-vip/,2025001000280002,28/10/2026,TRUE
2,zcrm_3584690000572508070,28/10/2025,Mikhail,Kulakov,kulakov130620@ya.ru,,6282341820854,,,Indonesia,Karma Resorts,Karma Beach,Karma Beach VIP 2023,Karma Beach VIP membership sign up 2023,,,https://karmagroup.com/karma-beach-vip/,2025001000280001,28/10/2026,TRUE
3,zcrm_3584690000572538001,28/10/2025,Luciana,Begazo,collabsluciana@gmail.com,,34613463996,,,Spain,Karma Resorts,Karma Beach,Karma Beach VIP 2023,Karma Beach VIP membership sign up 2023,,,https://karmagroup.com/karma-beach-vip/,2025001000280003,28/10/2026,FALSE
4,zcrm_3584690000572318166,27/10/2025,Valentina,Vrech,valentinavrech@hotmail.com,,31653858650,,,Netherlands,Karma Resorts,Karma Beach,Karma Beach VIP 2023,Karma Beach VIP membership sign up 2023,,,https://karmagroup.com/karma-beach-vip/,2025001000270001,27/10/2026,FALSE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10393,zcrm_3584690000007840074,02/04/2019,Oreste,Lentidoro,lentidoro@gmail.com,,61410699788,,,Australia,Karma Resorts,Karma Beach,Karma Beach VIP 2023,Karma Beach VIP membership sign up 2023,,Other - Other,https://karmagroup.com/karma-beach-vip/,2024000300100005,10/03/2025,FALSE
10394,zcrm_3584690000008059311,02/04/2019,Meylina,Ardiningsih,meylinaardi@gmail.com,,6281385951854,,,Indonesia,Karma Resorts,Karma Beach,Karma Beach VIP 2023,Karma Beach VIP membership sign up 2023,,Other - Other,https://karmagroup.com/karma-beach-vip/,2024000100200010,20/01/2025,FALSE
10395,zcrm_3584690000008362510,02/04/2019,Caslin,Liu,Caslinliu@yahoo.com.sg,,6591192897,,,Singapore,Karma Resorts,Other,Past Guests,Karma Beach VIP membership sign up 2023,Karma Kandara,Other Bali DB - Purchased,https://karmagroup.com/karma-beach-vip/,2024000300240002,24/03/2025,FALSE
10396,zcrm_3584690000007896028,02/04/2019,Wesley,Powell,wez.powell@gmail.com,,61459554763,,,Australia,Karma Resorts,Karma Beach,Karma Beach VIP 2023,Karma Beach VIP membership sign up 2023,,Other - Other,https://karmagroup.com/karma-beach-vip/,2024000500240018,24/05/2025,FALSE


In [12]:
# Concatenate DataFrames
df_final = pd.concat([df_update, df_live], ignore_index=True)

# Replace 'empty' with ''
df_final.replace('empty', '', inplace=True)

# Convert 'Do you live in Bali?' to uppercase
df_final['Do you live in Bali?'] = df_final['Do you live in Bali?'].astype(str).str.upper()

# Convert 'Created Time' to datetime format
df_final['Created Time'] = pd.to_datetime(df_final['Created Time'], dayfirst=True, errors='coerce')

# Sort by 'Created Time' in descending order
df_final.sort_values('Created Time', ascending=False, inplace=True)

# Format 'Created Time' back to the desired string format
df_final['Created Time'] = df_final['Created Time'].dt.strftime('%d/%m/%Y')

df_final

,Record Id,Created Time,First Name,Last Name,Email,Birthdate,Phone,Street,City,Country,Lead Brand,Lead Sub-Brand,Lead Source,Lead Source Description,Lead Locations,Lead History,Website,Card Number,Card Expired,Do you live in Bali?
0,zcrm_3584690000572744001,30/10/2025,Artha,Wanodya,arthawanodya@gmail.com,,6282182422800,,,Indonesia,Karma Resorts,Karma Beach,Karma Beach VIP 2023,Karma Beach VIP membership sign up 2023,,,https://karmagroup.com/karma-beach-vip/,2025001000300003,30/10/2026,FALSE
2,zcrm_3584690000572709001,30/10/2025,Natallia,Chumakova,go@royaltravel.by,,375293553464,,,Belarus,Karma Resorts,Karma Beach,Karma Beach VIP 2023,Karma Beach VIP membership sign up 2023,,,https://karmagroup.com/karma-beach-vip/,2025001000300001,30/10/2026,FALSE
1,zcrm_3584690000572709031,30/10/2025,Alex,Mcconnell,alextwine22@gmail.com,,61430296825,,,Australia,Karma Resorts,Karma Beach,Karma Beach VIP 2023,Karma Beach VIP membership sign up 2023,,,https://karmagroup.com/karma-beach-vip/,2025001000300002,30/10/2026,FALSE
3,zcrm_3584690000572567001,28/10/2025,Lino,Scidone,lino.scidone@gmail.com,,61425757400,,,Australia,Karma Resorts,Karma Beach,Karma Beach VIP 2023,Karma Beach VIP membership sign up 2023,,,https://karmagroup.com/karma-beach-vip/,2025001000280004,28/10/2026,FALSE
6,zcrm_3584690000572538001,28/10/2025,Luciana,Begazo,collabsluciana@gmail.com,,34613463996,,,Spain,Karma Resorts,Karma Beach,Karma Beach VIP 2023,Karma Beach VIP membership sign up 2023,,,https://karmagroup.com/karma-beach-vip/,2025001000280003,28/10/2026,FALSE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10377,zcrm_3584690000007207443,02/04/2019,Cheryl,Macdonald,cheryl@yogabellies.co.uk,21/08/1979,6586066573,"the seaview, 37 amber road",Singapore,Singapore,Timeshare Marketing,Karma Club,Bamboo,Karma Beach VIP membership sign up 2023,,Karma Club - Bamboo,https://karmagroup.com/karma-beach-vip/,2024000100290008,29/01/2025,FALSE
10376,zcrm_3584690000007204035,02/04/2019,Sri,Ratna,ratna_pink@yahoo.com,24/06/1986,6597108106,06-438 PUNGGOL WALK,Singapore,Singapore,Timeshare Marketing,Karma Club,Membership,Karma Beach VIP membership sign up 2023,,Karma Club - Membership,https://karmagroup.com/karma-beach-vip/,2024001000250004,25/10/2025,FALSE
10375,zcrm_3584690000006037781,02/04/2019,Nadine,Samir,nadinesamir1@gmail.com,,61412990928,40 Farivell St,Hunters Hill Nsw,Australia,Karma Resorts,Other,Past Guests,Karma Beach VIP membership sign up 2023,Karma Kandara,"Other - Past Guests, Other - Web Chat, Other -...",https://karmagroup.com/karma-beach-vip/,2024000700080003,08/07/2025,FALSE
10374,zcrm_3584690000005974937,02/04/2019,Paula,mcgeown,paulamcgeown@gmail.com,,61481151026,,,Australia,Karma Resorts,Other,Past Guests,Karma Beach VIP membership sign up 2023,Karma Rottnest,"Other - Past Guests, BGAU - Prospect",https://karmagroup.com/karma-beach-vip/,2025000100170001,17/01/2026,FALSE


In [13]:
df_live.columns
# new_df.columns

Index(['Record Id', 'Created Time', 'First Name', 'Last Name', 'Email',
       'Birthdate', 'Phone', 'Street', 'City', 'Country', 'Lead Brand',
       'Lead Sub-Brand', 'Lead Source', 'Lead Source Description',
       'Lead Locations', 'Lead History', 'Website', 'Card Number',
       'Card Expired', 'Do you live in Bali?'],
      dtype='object')

In [14]:
# Function to sanitize DataFrame
def sanitize_dataframe(df):
    # Replace invalid float values with None
    df = df.applymap(lambda x: None if isinstance(x, float) and (np.isinf(x) or np.isnan(x)) else x)
    return df

# Sanitize the DataFrame
df_final = sanitize_dataframe(df_final)
df_final['Card Number'] = df_final['Card Number'].astype(str)
df_final.drop(df_final.loc[df_final['Email'].str.contains('karmagroup.com', regex=True, flags=re.I)].index, inplace=True)
df_final

C:\Users\fajar.fatoni\AppData\Local\Temp\ipykernel_20312\4224994593.py:4: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: None if isinstance(x, float) and (np.isinf(x) or np.isnan(x)) else x)


,Record Id,Created Time,First Name,Last Name,Email,Birthdate,Phone,Street,City,Country,Lead Brand,Lead Sub-Brand,Lead Source,Lead Source Description,Lead Locations,Lead History,Website,Card Number,Card Expired,Do you live in Bali?
0,zcrm_3584690000572744001,30/10/2025,Artha,Wanodya,arthawanodya@gmail.com,,6282182422800,,,Indonesia,Karma Resorts,Karma Beach,Karma Beach VIP 2023,Karma Beach VIP membership sign up 2023,,,https://karmagroup.com/karma-beach-vip/,2025001000300003,30/10/2026,FALSE
2,zcrm_3584690000572709001,30/10/2025,Natallia,Chumakova,go@royaltravel.by,,375293553464,,,Belarus,Karma Resorts,Karma Beach,Karma Beach VIP 2023,Karma Beach VIP membership sign up 2023,,,https://karmagroup.com/karma-beach-vip/,2025001000300001,30/10/2026,FALSE
1,zcrm_3584690000572709031,30/10/2025,Alex,Mcconnell,alextwine22@gmail.com,,61430296825,,,Australia,Karma Resorts,Karma Beach,Karma Beach VIP 2023,Karma Beach VIP membership sign up 2023,,,https://karmagroup.com/karma-beach-vip/,2025001000300002,30/10/2026,FALSE
3,zcrm_3584690000572567001,28/10/2025,Lino,Scidone,lino.scidone@gmail.com,,61425757400,,,Australia,Karma Resorts,Karma Beach,Karma Beach VIP 2023,Karma Beach VIP membership sign up 2023,,,https://karmagroup.com/karma-beach-vip/,2025001000280004,28/10/2026,FALSE
6,zcrm_3584690000572538001,28/10/2025,Luciana,Begazo,collabsluciana@gmail.com,,34613463996,,,Spain,Karma Resorts,Karma Beach,Karma Beach VIP 2023,Karma Beach VIP membership sign up 2023,,,https://karmagroup.com/karma-beach-vip/,2025001000280003,28/10/2026,FALSE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10377,zcrm_3584690000007207443,02/04/2019,Cheryl,Macdonald,cheryl@yogabellies.co.uk,21/08/1979,6586066573,"the seaview, 37 amber road",Singapore,Singapore,Timeshare Marketing,Karma Club,Bamboo,Karma Beach VIP membership sign up 2023,,Karma Club - Bamboo,https://karmagroup.com/karma-beach-vip/,2024000100290008,29/01/2025,FALSE
10376,zcrm_3584690000007204035,02/04/2019,Sri,Ratna,ratna_pink@yahoo.com,24/06/1986,6597108106,06-438 PUNGGOL WALK,Singapore,Singapore,Timeshare Marketing,Karma Club,Membership,Karma Beach VIP membership sign up 2023,,Karma Club - Membership,https://karmagroup.com/karma-beach-vip/,2024001000250004,25/10/2025,FALSE
10375,zcrm_3584690000006037781,02/04/2019,Nadine,Samir,nadinesamir1@gmail.com,,61412990928,40 Farivell St,Hunters Hill Nsw,Australia,Karma Resorts,Other,Past Guests,Karma Beach VIP membership sign up 2023,Karma Kandara,"Other - Past Guests, Other - Web Chat, Other -...",https://karmagroup.com/karma-beach-vip/,2024000700080003,08/07/2025,FALSE
10374,zcrm_3584690000005974937,02/04/2019,Paula,mcgeown,paulamcgeown@gmail.com,,61481151026,,,Australia,Karma Resorts,Other,Past Guests,Karma Beach VIP membership sign up 2023,Karma Rottnest,"Other - Past Guests, BGAU - Prospect",https://karmagroup.com/karma-beach-vip/,2025000100170001,17/01/2026,FALSE


In [15]:
# Update keseluruhan data, delete data yang lama terlebih dahulu
# clear all row before update
worksheet.clear()
worksheet.update([df_final.columns.values.tolist()] + df_final.values.tolist())

{'spreadsheetId': '1_zOrWK1gAo-tHDsKqU6AaqVzi4xaNpOELDwtbBx5F6M',
 'updatedRange': "'Karma Beach - LIVE'!A1:T10402",
 'updatedRows': 10402,
 'updatedColumns': 20,
 'updatedCells': 208040}

In [16]:
print("Data update kbb vip : "+str(count_df_update))

Data update kbb vip : 3
